In [5]:
from paddlenlp.transformers import BertModel, BertTokenizer,BertForMaskedLM
import paddle

In [2]:
model = BertModel.from_pretrained('bert-base-chinese')
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

[2022-05-18 19:13:27,083] [    INFO] - Already cached /home/ricardo/.paddlenlp/models/bert-base-chinese/bert-base-chinese.pdparams
[2022-05-18 19:14:19,257] [    INFO] - Weights from pretrained model not used in BertModel: ['cls.predictions.decoder_weight', 'cls.predictions.decoder_bias', 'cls.predictions.transform.weight', 'cls.predictions.transform.bias', 'cls.predictions.layer_norm.weight', 'cls.predictions.layer_norm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
[2022-05-18 19:14:20,148] [    INFO] - Already cached /home/ricardo/.paddlenlp/models/bert-base-chinese/bert-base-chinese-vocab.txt


In [10]:
test_str1 = '恒大地产2012年拟裁员30%：1月4日，恒大地产（3333.HK）的一位副总裁向记者确认，公司内部发文，2012年将裁员30%。知情人士透露，按步骤初期只裁10%，因为2009年裁员的时候，矛盾冲突比较激烈。该副总称裁员的原因主要是项目销售问题。'
model_inputs1 = tokenizer(test_str1)
model_outputs1 = model(**{k:paddle.to_tensor([v]) for (k,v) in model_inputs1.items()})

In [15]:
print(model_inputs1)

{'input_ids': [101, 2608, 1920, 1765, 772, 8151, 2399, 2877, 6161, 1447, 8114, 110, 8038, 122, 3299, 125, 3189, 8024, 2608, 1920, 1765, 772, 8020, 10745, 8152, 119, 100, 8021, 4638, 671, 855, 1199, 2600, 6161, 1403, 6381, 5442, 4802, 6371, 8024, 1062, 1385, 1079, 6956, 1355, 3152, 8024, 8151, 2399, 2199, 6161, 1447, 8114, 110, 511, 4761, 2658, 782, 1894, 6851, 7463, 8024, 2902, 3635, 7752, 1159, 3309, 1372, 6161, 8108, 110, 8024, 1728, 711, 8170, 2399, 6161, 1447, 4638, 3198, 952, 8024, 4757, 4688, 1103, 4960, 3683, 6772, 4080, 4164, 511, 6421, 1199, 2600, 4917, 6161, 1447, 4638, 1333, 1728, 712, 6206, 3221, 7555, 4680, 7218, 1545, 7309, 7579, 511, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [27]:
pos_list1 = [
    [18,28],
    [103,105],
    [105,109],
    [20,22],
]
for pos in pos_list1:
    print(tokenizer.convert_ids_to_tokens(model_inputs1['input_ids'][pos[0]:pos[1]]))

['恒', '大', '地', '产', '（', '333', '##3', '.', '[UNK]', '）']
['项', '目']
['销', '售', '问', '题']
['地', '产']


In [28]:
test_str2 = '从2021年9月份开始，恒大全国各地项目接连停工率，交楼量不足1万套导致恒大的销量大幅下降；每天都有商票逾期，债券经被停止融资融业务，裁员60%'
model_inputs2 = tokenizer(test_str2)
model_outputs2 = model(**{k:paddle.to_tensor([v]) for (k,v) in model_inputs2.items()})

In [40]:
print(model_outputs2[0])

Tensor(shape=[1, 70, 768], dtype=float32, place=CPUPlace, stop_gradient=False,
       [[[ 0.81039554,  0.15714262, -0.33748814, ..., -0.82522547,
           0.03717040,  0.39957264],
         [ 0.18499802,  0.41343406,  1.15910769, ..., -0.65995526,
          -0.32494196,  0.22273116],
         [ 0.66050130,  0.17841768,  1.48442113, ...,  0.11918050,
           0.87924564, -0.08119622],
         ...,
         [-0.75124550, -0.86865503, -0.64447254, ..., -0.53708994,
          -0.03471069,  0.24607730],
         [ 1.64574575, -0.56429189,  1.26031041, ..., -0.29384169,
          -0.11994778,  0.50716025],
         [ 0.45596373,  0.08586539,  1.00020325, ..., -0.57331502,
          -0.22262847, -0.14491002]]])


In [45]:
pos_list2 = [
    [34,43],
    [10,12],
    [18,22],
    [10,12],
]
for pos in pos_list2:
    print(tokenizer.convert_ids_to_tokens(model_inputs2['input_ids'][pos[0]:pos[1]]))

entity_embs2 = []
for pos in pos_list2:
    temp_maxtrix = paddle.stack([model_outputs2[0][0,idx] for idx in range(pos[0],pos[1])])
    entity_embs2.append(paddle.max(temp_maxtrix,axis=0))
    

['恒', '大', '的', '销', '量', '大', '幅', '下', '降']
['恒', '大']
['接', '连', '停', '工']
['恒', '大']


4

In [ ]:
x = paddle.to_tensor([[0.2, 0.3, 0.5, 0.9],
                      [0.1, 0.2, 0.6, 0.7]],
                     dtype='float64', stop_gradient=False)

In [38]:
print(paddle.max(x,axis=0))

Tensor(shape=[4], dtype=float64, place=CPUPlace, stop_gradient=False,
       [0.20000000, 0.30000000, 0.60000000, 0.90000000])
